# Using Historical Data to Predict Batting Success: Step 5 of 5

Authored by: Donna J. Harris (994042890)

Email: harr2890@mylaurier.ca

For: CP640 Machine Learning (S22) with Professor Elham Harirpoush

## Notebook Series

Just a word about the presentation of this project code.

The code is organized into a series of locally executed Jupyter notebooks, organized by step and needing to be executed in sequence to follow the flow of the entire project.

This is `harr2890_project_step5_ops_modelling`, the fifth of five notebooks.

## *Step 5 - Exploration and Modelling for an OPS Approach*

Baseball is full of statistics but one of the clearest statistics about a batter's effectiveness at the plate is the OPS, which is the sum of the player's On Base Percentage (an important indication of how often they get on base) and the player's Slugging Percentage (an indicator of the hitting power a player posesses, which can point to their ability to help their on-base teammates to score).

This approach for predicting batting success goes directly to this statistic and aims to explore how past data might have hints about a player's future with respect to the OPS statistic.

Generally speaking, a player with an OPS above 0.7666 is above average, with higher values classifying the player as increasingly excellent.  (Reference: https://en.wikipedia.org/wiki/On-base_plus_slugging)

## Environment Setup

Import and establish environment for our work, including showing all dataframe column values.

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

### Pre-Conditions

Steps 1 and 4 must be run completely before running this notebook.

The `data` folder must exist with the following prepared data file:
- `./data/step4_ops_data.csv`

##  Loading Prepared Data Files

In [ ]:
alldata_csv = "./data/step4_ops_data.csv"
all_data = pd.read_csv(alldata_csv)
all_data

### Examining Relationships Between Features

The data used in this approach is much more straightforward than in the Hall of Fame Approach. We're looking at past OPS values to predict future OPS values. But we want to know which ones to pick.

In [ ]:
import matplotlib.pyplot as plt

def scatter_plot(feature, target):
    plt.figure(figsize=(10, 8))
    plt.scatter(
        all_data[feature],
        all_data[target],
        c='lightgreen',        
        edgecolors=(0, 0, 0)
    )
    
    plt.xlim(0.0, 5.2)
#     plt.ylim(0.0, 1.4)


    plt.xlabel("{}".format(feature))
    plt.ylabel("{}".format(target))
    plt.show()

First, we'll see what we should use to predict the OPS of a player's sixth season, given they have played five seasons already.

In [ ]:
scatter_plot('OPS Y1', 'OPS Y6')
scatter_plot('OPS Y2', 'OPS Y6')
scatter_plot('OPS Y3', 'OPS Y6')
scatter_plot('OPS Y4', 'OPS Y6')
scatter_plot('OPS Y5', 'OPS Y6')

We can see there is a tighter line forming the closer we get to the sixth year. In particular, years three through five are of interest.

Next we'll look at the first five seasons with respect to the tenth season's OPS.

In [ ]:
scatter_plot('OPS Y1', 'OPS Y10')
scatter_plot('OPS Y2', 'OPS Y10')
scatter_plot('OPS Y3', 'OPS Y10')
scatter_plot('OPS Y4', 'OPS Y10')
scatter_plot('OPS Y5', 'OPS Y10')

Here, we see a similar pattern with the OPS values from seasons closer to the tenth season seem to have more definition, however it isn't as tightly formed in the plots as with the sixth season.

Finally, we'll look for relationships with the overall career OPS of a player. (Keeping in mind that this represents a value for both completed and in-progress careers.)

In [ ]:
scatter_plot('OPS Y1', 'Career OPS')
scatter_plot('OPS Y2', 'Career OPS')
scatter_plot('OPS Y3', 'Career OPS')
scatter_plot('OPS Y4', 'Career OPS')
scatter_plot('OPS Y5', 'Career OPS')

We see much tigheter lines forming up for the career OPS values, especially for the third, fourth, and fifth sesaons, than we do for the others.

I'm surprised by these results. I anticipated the 6th season OPS and/or the 10th season OPS value to be more strongly correalated than the Career OPS, but it looks like Career OPS is more strongly related. I thought it would be stronger between consecutive seasons than this tends to indicate.

Let's build three models, both with the 3rd, 4th, and 5th season OPS values as features.

Model 1 (M1) will aim to predict the 6th season OPS.
Model 2 (M2) will aim to predict the 10th season OPS.
Model 3 (M3) will aim to predict the player's Career OPS.

Based on these plots, we might expect to see the most accuracy with M3. Let's try and see.

## Data Preparation

First, we'll build the dataframes, features, and targets we need. This will be used as X for all models.

In [ ]:
X = all_data[['OPS Y3', 'OPS Y4', 'OPS Y5']]
X

In [ ]:
X = X.values
X

Next, we need three target array, which will be numbered to make the model they correspond with.

The target array for `'OPS Y6'` (Model 1) is `y1`.

In [ ]:
y1 = all_data['OPS Y6']
y1 = y1.values
y1

The target array for `'OPS Y10'` (Model 2) is `y2`.

In [ ]:
y2 = all_data['OPS Y10']
y2 = y2.values
y2

The target array for `'Career OPS'` (Model 3) is `y3`.

In [ ]:
y3 = all_data['Career OPS']
y3 = y3.values
y3

## Building Models

The following is a model evaluation function that will be used throughout the notebook.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import matplotlib.pyplot as plt

def print_model_evaluation(name, y, actual, predicted):
    
    mae = mean_absolute_error(actual, predicted)
    rmse = (np.sqrt(mean_squared_error(actual, predicted)))
    r2 = r2_score(actual, predicted)

    print("Model Performance\n")

    print('MAE\t= %.3f' % (mae))
    print('RMSE\t= %.3f' % (rmse))
    print('R^2\t= %.3f' % (r2))

    fig, ax = plt.subplots()
    ax.scatter(actual, predicted, edgecolors=(0, 0, 0))
    ax.plot([y.min(), y.max()], [y.min(), y.max()], "k--", lw=1)
    title = "Measured vs. Predicted\n(" + name + ")"
    ax.set_title(title)
    ax.set_xlabel("Measured")
    ax.set_ylabel("Predicted")
    plt.show()

In [ ]:
def store_r2(model_name, r2, results):
    results.append([model_name, r2])
    return results

### Model 1 (M1) - Predicting Y6

We'll split the data into three sets for this approach, training, validation, and testing.

Then, create our linear regression model using `'OPS Y3'`, `'OPS Y4'`, and `'OPS Y5'` to predict `'OPS Y6'`.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size = 0.3, random_state = 42)
X_validate, X_test, y1_validate, y1_test = train_test_split(X_test, y1_test, test_size = 0.5, random_state = 42)

from sklearn.linear_model import LinearRegression

M1 = LinearRegression()
M1.fit(X_train, y1_train)

y1_predict_validate = M1.predict(X_validate)

print_model_evaluation('OPS Y6', y1, y1_validate, y1_predict_validate)

### Model 2 (M2) - Predicting Y10

We'll split the data into three sets for this approach, training, validation, and testing.

Then, create our linear regression model using `'OPS Y3'`, `'OPS Y4'`, and `'OPS Y5'` to predict `'OPS Y10'`.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size = 0.3, random_state = 42)
X_validate, X_test, y2_validate, y2_test = train_test_split(X_test, y2_test, test_size = 0.5, random_state = 42)

from sklearn.linear_model import LinearRegression

M2 = LinearRegression()
M2.fit(X_train, y2_train)

y2_predict_validate = M2.predict(X_validate)

print_model_evaluation('OPS Y10', y2, y2_validate, y2_predict_validate)

### Model 3 (M3) - Predicting Career OPS

We'll split the data into three sets for this approach, training, validation, and testing.

Then, create our linear regression model using `'OPS Y3'`, `'OPS Y4'`, and `'OPS Y5'` to predict `'Career OPS'`.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y3_train, y3_test = train_test_split(X, y3, test_size = 0.3, random_state = 42)
X_validate, X_test, y3_validate, y3_test = train_test_split(X_test, y3_test, test_size = 0.5, random_state = 42)

from sklearn.linear_model import LinearRegression

M3 = LinearRegression()
M3.fit(X_train, y3_train)

y3_predict_validate = M3.predict(X_validate)

print_model_evaluation('Career OPS', y3, y3_validate, y3_predict_validate)

After observing the results of the initial three models, we can see a very strong linear correalation but also the strongest predictor is clear. For the remainder of this approach, we will continue to attempt to develop a model which will predict career batting success based on third, fourth, and fifth year OPS statistics.

## Improving M3 - Predicting Career OPS

Let's capture the R-Squared values for our M3 variants in `M3_results`, then try to improve on the M3 prediction approach.

In [ ]:
M3_results = []
M3_results = store_r2("Lin Reg", r2_score(y3_validate, y3_predict_validate), M3_results)

### M3 - SGDRegressor

Looking first at a Stochastic Gradient Descent regressor, we'll run `GridSearchCV` first to see if we can locate any some helpful parameters for tuning, before training this variant of our model.

In [ ]:
# Reference: https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor

tuned_parameters = [
    {"learning_rate": ["optimal"], "alpha": [0.1, 0.01, 1e-3, 1e-4], "max_iter": [2000]}
]

scores = ["neg_mean_absolute_error", "neg_root_mean_squared_error", "r2"]

for score in scores:
    print("#### Tuning Hyper-Parameters for",score)
    print()

    clf = GridSearchCV(SGDRegressor(), tuned_parameters, scoring="%s" % score)
    clf.fit(X, y3)

    print("Best parameters on dataset (X):",clf.best_params_)
    print("\nGrid scores on dataset (X):")
    means = clf.cv_results_["mean_test_score"]
    stds = clf.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
        print("\t%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print("\n\n")


Now, to use the identified parameters.

In [ ]:
from sklearn.linear_model import SGDRegressor

M3_SDG = SGDRegressor(learning_rate='optimal', alpha=0.001, max_iter=2000)
M3_SDG.fit(X_train, y3_train)

y3_predict_validate = M3_SDG.predict(X_validate)

print_model_evaluation('Career OPS', y3, y3_validate, y3_predict_validate)

This isn't much different, but we'll add it to our results.

In [ ]:
M3_results = store_r2("SDG Reg", r2_score(y3_validate, y3_predict_validate), M3_results)
M3_results

Let's try cross-validation:

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(M3_SDG, X, y3, cv=50, scoring='r2')
scores['test_score']

In [ ]:
np.mean(scores['test_score'])

In [ ]:
M3_results = store_r2("SDG +CV", np.mean(scores['test_score']), M3_results)

Not much different here either.

### M3 - Gradient Boosting

Next is Gradient Boosting. This one is a good candidate for improvements.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

M3_GradBoost = GradientBoostingRegressor()
M3_GradBoost.fit(X_train, y3_train)

y3_predict_validate = M3_GradBoost.predict(X_validate)

print_model_evaluation('Career OPS', y3, y3_validate, y3_predict_validate)

The R-squared value has increased a little and the error values remain low.

In [ ]:
M3_results = store_r2("GradBoost", r2_score(y3_validate, y3_predict_validate), M3_results)

Let's cross validate, as well.

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(M3_GradBoost, X, y3, cv=50, scoring='r2')
scores['test_score']

In [ ]:
np.mean(scores['test_score'])

Not a big change, but a small increase.

In [ ]:
M3_results = store_r2("GradBoost +CV", np.mean(scores['test_score']), M3_results)

### M5 - Grad Boost w/ K-Fold Validation

Returning to Gradient Boosting, we'll try with K-fold validation.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

M3_GradBoost_K = GradientBoostingRegressor()
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

n_scores = cross_val_score(M3_GradBoost_K, X_train, y3_train, scoring='r2', cv=10, n_jobs=-1, error_score='raise')

M3_GradBoost_K.fit(X_train, y3_train)

y3_predict_validate = M3_GradBoost_K.predict(X_validate)

print_model_evaluation('Career OPS', y3, y3_validate, y3_predict_validate)

About the same as without, which seems to be the trend with our very linear data.

In [ ]:
M3_results = store_r2("GradBoost_K", r2_score(y3_validate, y3_predict_validate), M3_results)

And cross-validating,

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(M3_GradBoost_K, X, y3, cv=50, scoring='r2')
scores['test_score']

In [ ]:
np.mean(scores['test_score'])

Very much in the same vein once again.

In [ ]:
M3_results = store_r2("GradBoostK +CV", np.mean(scores['test_score']), M3_results)

### M3 - SVR with GridSearchCV results

Looking in a different direction with a Support Vector Machine, but first using `GridSearchCV`.

In [ ]:
# Reference: https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

tuned_parameters = [
    {"kernel": ["rbf"], "gamma": [1e-3, 1e-4], "C": [1, 10, 100, 1000]},
    {"kernel": ["linear"], "C": [1, 10, 100, 1000]},
]

scores = ["neg_mean_absolute_error", "neg_root_mean_squared_error", "r2"]

for score in scores:
    print("#### Tuning Hyper-Parameters for",score)
    print()

    clf = GridSearchCV(SVR(), tuned_parameters, scoring="%s" % score)
    clf.fit(X, y3)

    print("Best parameters on dataset (X):",clf.best_params_)
    print("\nGrid scores on dataset (X):")
    means = clf.cv_results_["mean_test_score"]
    stds = clf.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
        print("\t%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print("\n\n")


And then using the 'best' parameters identified in our search.

In [ ]:
from sklearn.svm import SVR

M3_SVR_Grid = SVR(kernel='rbf', gamma=0.001, C=1000)
M3_SVR_Grid.fit(X_train, y3_train)

y3_predict_validate = M3_SVR_Grid.predict(X_validate)

print_model_evaluation('Career OPS', y3, y3_validate, y3_predict_validate)

These results are higher than most so far, but not better than GradientBoosting.

In [ ]:
M3_results = store_r2("SVR_Grid", r2_score(y3_validate, y3_predict_validate), M3_results)

Cross-validating...

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(M3_SVR_Grid, X, y3, cv=50, scoring='r2')
scores['test_score']

In [ ]:
np.mean(scores['test_score'])

And, once again, no big changes here.

In [ ]:
M3_results = store_r2("SVR_Grid +CV", np.mean(scores['test_score']), M3_results)

### Summary of M3 variants

To recap, here are the models and also the mean R-squared value after cross validating (+CV).

In [ ]:
for row in M3_results:
    print(row[0],"\tR^2 = ",row[1])

The GradientBoosting regressors provided the most, albeit marginal, improvement to our models and the R-squared value.

## Model Evaluation with Test Data

Now that we have some models with somewhat improved R-scores, we can make use of the `y3_test` data that was split up to evaluate the models with totally unseen data.

GradBoosting and GradBoosting with KFold validation are pretty close, so we'll take a quick look at both.

### M3 using GradBoost

In [ ]:
y3_predict_test = M3_GradBoost.predict(X_test)

print_model_evaluation('Career OPS', y3, y3_test, y3_predict_test)

In [ ]:
y3_predict_test = M3_GradBoost_K.predict(X_test)

print_model_evaluation('Career OPS', y3, y3_test, y3_predict_test)

Again, these results remain pretty similar to one another and also, although slightly lower than our validation scores, is still relatively close to what we saw earlier. There isn't much variance, which is good.

## Model Evaluation Based on Context

How good is our selected model at correctly identifying _**above average**_ hitters, or better? If we consider the measure mentioned in the introduction, then one thought is looking at if the model is _generally_ good at identifying if a player is above average, or better.

So, in this case, the question becomes less about what the specific value of the OPS is and more about if the player is predicted correctly as being above average (based on predicted Career OPS) or not.

This last function is a test looking for the number of accurately classified players, in this general sense. For example, if a batter's predicted OPS is above average but their actual OPS is below average, then this would not be considered a correct prediction.

In [ ]:
def general_classification_accuracy(actual, prediction):
    if len(actual) == len(prediction):
        i = 0
        correct_classifications = 0
        
        while i < len(actual):
            actual_ops = actual[i]
            predicted_ops = prediction[i]

            if ((actual_ops > 0.7666) and (predicted_ops > 0.7666)) or \
                ((actual_ops <= 0.7666) and (predicted_ops <= 0.7666)):
                correct_classifications += 1
                
            i+=1
                
    return correct_classifications/(len(actual)*1.0)


general_classification_accuracy(y3_test, y3_predict_test)

What this metric shows us is that the model correctly identifies players as either above or below average 91.1% of the time.

While it isn't a deeply meaningful metric, it is worth examining since it both confirms the model as _generally_ accurate model for this problem. Generally speaking, this appears to be an adequate model, albeit a very naive one in the larger world of baseball data and sabermetrics.

## Comparison of the Hall of Fame Approach and the OPS Approach

The OPS Approach is generally a good indicator of future batting success over a player's career, but not a stellar one. It is definitely more reliable than the Hall of Fame Approach for predicting batting success and is far more tangible in nature.

The challenge with the Hall of Fame Approach is the very complicated world of selection and voting and things not on paper, statistically speaking, that play into whether or not a player is inducted. While at first thought it seemed like a valuable approach (after all, there aren't too many mediocre batters in the Hall of Fame) it was thoroughly flawed and not generally helpful for predicting those who will have moderate batting success throughout their careers but will not end up as Hall of Fame superstars.

The surprise with the OPS Approach was how the strongest approach was to predict the Career OPS and not an OPS for a season that was more near-term in the future. The challenge with this approach is likely that of the many additional dynamics of play that it does not account for, even with respect to batting. For instance, a player may not have a high OPS but still be a very effective batter that contributes well to the team. This model would not be able to predict those player's successes at the plate because we were examining too specific of a measure. However, overall, this is a valuable, albeit imperfect, statistic for general use.